In [1]:
from langchain.document_loaders import UnstructuredURLLoader


In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [3]:
from transformers import pipeline

/opt/conda/envs/newsummary/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [6]:
from transformers import BartTokenizerFast

In [7]:
print("calling tokenizer....")
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
print("calling model.....")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")
print("model loaded....")

def get_summary(final_text):
  pipe_sum = pipeline(
        'summarization',
        model = model,
        tokenizer = tokenizer
        )
  result = pipe_sum(final_text,
                    max_length = 1024, 
                    min_length = 100,
                    do_sample=False,
                    truncation=True)
  result = result[0]['summary_text']
  
  return result

def text_preprocessing(u):
  loaders = UnstructuredURLLoader(u)
  data = loaders.load()
  print(len(data))
  text_splitter = RecursiveCharacterTextSplitter(
      chunk_size=1000,
      chunk_overlap=200
      )
  # As data is of type documents we can directly use split_documents over split_text in order to get the chunks.
  docs = text_splitter.split_documents(data)
  print(len(docs))
  final_texts = ""
  for i in range(0,len(docs)):
    #print(i," : ", input_text[i].page_content)
    final_texts = final_texts + docs[i].page_content
    #print("final_texts : ", final_texts)
  return final_texts

urls_1=[
    "https://www.news.com.au/world/coronavirus/health/wear-a-mask-nsw-health-responds-to-a-rise-in-cases-in-light-of-new-subvariant-strains/news-story/90cad04f2a329d8730871c00b3dd00cc"
    ]

# for i in range(0,len(urls)):
#   print(urls[i])
#   r = text_preprocessing(urls[i])

final_text_1 = text_preprocessing(urls_1)
#get_summary( final_text_1)
print("Summary 1: ",get_summary(final_text_1))

urls_2 =[
    "https://www.news.com.au/finance/business/other-industries/liquidator-of-collapsed-building-firm-alleges-director-was-loaned-nearly-1-million-in-company-money/news-story/493daf5d91f6a6b1267d542d6fa5b184"
    ]
final_text_2 = text_preprocessing(urls_2)
print("Summary 2: ",get_summary(final_text_2))

calling tokenizer....
calling model.....
model loaded....
1
11
Summary 1:  Two new subvariants of Covid are ripping through multiple states, forcing health officials to call for masks to stem transmission. Residents across one state have been urged to wear a mask if they show Covid symptoms as a new wave of infection sweeps parts of the country. New South Wales has recorded its highest level of the virus in a year, with more than 17 per cent of PCR tests returning positive results. NSW Health is yet to release their report for the New Year, with increasing numbers of cases across the state.
1
13
Summary 2:  Liquidator of collapsed building firm alleges director was loaned nearly $1 million in company money according to public liquidator’s report. The regulator, ASIC, has not taken action against the director under the Corporations Act.Iconic Aussie retailer to close forever. Selena Gomez’S $3 billion beauty move. The Australian dollar dips before blockbuster US meeting. The Aussie doll

In [8]:
# print("calling tokenizer....")
# tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
# print("calling model.....")
# model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")
# print("model loaded....")

def get_summary(final_text):
#   pipe_sum = pipeline(
#         'summarization',
#         model = model,
#         tokenizer = tokenizer
#         )
  summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

  result = summarizer(final_text,
                    max_length = 1024, 
                    min_length = 100,
                    do_sample=False,
                    truncation=True)
  result = result[0]['summary_text']
  
  return result

def text_preprocessing(u):
  loaders = UnstructuredURLLoader(u)
  data = loaders.load()
  print(len(data))
  text_splitter = RecursiveCharacterTextSplitter(
      chunk_size=1000,
      chunk_overlap=200
      )
  # As data is of type documents we can directly use split_documents over split_text in order to get the chunks.
  docs = text_splitter.split_documents(data)
  print(len(docs))
  final_texts = ""
  for i in range(0,len(docs)):
    #print(i," : ", input_text[i].page_content)
    final_texts = final_texts + docs[i].page_content
    #print("final_texts : ", final_texts)
  return final_texts

urls_1=[
    "https://www.news.com.au/world/coronavirus/health/wear-a-mask-nsw-health-responds-to-a-rise-in-cases-in-light-of-new-subvariant-strains/news-story/90cad04f2a329d8730871c00b3dd00cc"
    ]

# for i in range(0,len(urls)):
#   print(urls[i])
#   r = text_preprocessing(urls[i])
print("URL :", type(urls_1), "  ", urls_1)
final_text_1 = text_preprocessing(urls_1)
#get_summary( final_text_1)
print("Summary 1: ",get_summary(final_text_1))

urls_2 =[
    "https://www.news.com.au/finance/business/other-industries/liquidator-of-collapsed-building-firm-alleges-director-was-loaned-nearly-1-million-in-company-money/news-story/493daf5d91f6a6b1267d542d6fa5b184"
    ]
final_text_2 = text_preprocessing(urls_2)
print("Summary 2: ",get_summary(final_text_2))

URL : <class 'list'>    ['https://www.news.com.au/world/coronavirus/health/wear-a-mask-nsw-health-responds-to-a-rise-in-cases-in-light-of-new-subvariant-strains/news-story/90cad04f2a329d8730871c00b3dd00cc']
1
11
Summary 1:  Two new subvariants of Covid are ripping through multiple states, forcing health officials to call for masks to stem transmission. Residents across one state have been urged to wear a mask if they show Covid symptoms as a new wave of infection sweeps parts of the country. New South Wales has recorded its highest level of the virus in a year, with more than 17 per cent of PCR tests returning positive results. NSW Health is yet to release their report for the New Year, with increasing numbers of cases across the state.
1
13
Summary 2:  Liquidator of collapsed building firm alleges director was loaned nearly $1 million in company money according to public liquidator’s report. The regulator, ASIC, has not taken action against the director under the Corporations Act.Icon